In [ ]:
import os, time
import numpy as np
import shutil
import random
import math
import torch
import sys
import imageio

curr_dir = os.path.dirname(os.path.abspath('__file__'))
root_dir = os.path.join(curr_dir, '..')
root_dir = os.path.join(root_dir, '..')
root_dir = "C:/evogym/evogym/examples"
external_dir = os.path.join(root_dir, 'externals')
sys.path.insert(0, root_dir)
sys.path.insert(1, os.path.join(external_dir, 'pytorch_a2c_ppo_acktr_gail'))

from ppo.envs import make_vec_envs
from ppo import utils

In [ ]:
torch.manual_seed(1)

controller_path = "demo/catcher/task-0_robot-19vae_controller.pt"
actor_critic, obs_rms = torch.load(controller_path, map_location='cpu')
structure_path = 'demo/catcher/task-0_struct-19vae.npz'

structure_data = np.load(structure_path)
structure = []

for key, value in structure_data.items():
    structure.append(value)
structure = tuple(structure)

env = make_vec_envs(
                'Catcher-v0',
                structure,
                5,
                1,
                None,
                None,
                device='cpu',
                allow_early_resets=False)

vec_norm = utils.get_vec_normalize(env)
if vec_norm is not None:
    vec_norm.eval()
    vec_norm.obs_rms = obs_rms

recurrent_hidden_states = torch.zeros(1, actor_critic.recurrent_hidden_state_size)
masks = torch.zeros(1, 1)

obs = env.reset()
env.render('screen')

total_steps = 0
reward_sum = 0
# imgs = []
while total_steps < 200000:
    with torch.no_grad():
        value, action, _, recurrent_hidden_states = actor_critic.act(
            obs, recurrent_hidden_states, masks, deterministic=True)


    # Obser reward and next obs
    obs, reward, done, _ = env.step(action)
    masks.fill_(0.0 if (done) else 1.0)
    reward_sum += reward
    total_steps += 1
    
    # img = env.render(mode='img')
    # imgs.append(img)
    
    env.render('screen')
    # env.get_attr("default_viewer", indices=None)[0].render("screen", hide_grid=True) # 隐藏背景网格
    
    if done==True:
        env.venv.close()
        break

env.venv.close()

In [ ]:
# imageio.mimsave(save_path, imgs, fps=1)